# GERMASum: GERman Mean Abstractive Summarization
## Abstractive Summarization of German News articles with Averaging LSTM States
### Training the SentenceClusterModel

In [1]:
#!pip uninstall -y torch
#!pip install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 torchtext==0.9.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
#!pip install -q cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install transformers datasets rouge langdetect sentence-transformers

import torch.nn.functional as F
import pandas as pd
import os
import numpy as np
import torch
import nltk
import time
import json
import pickle

from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from math import log
from tqdm import tqdm
from rouge import Rouge
from datetime import datetime, timezone
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from langdetect import detect

from modules.models import EncoderDecoder, Encoder, Decoder, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset
from modules.training import SummaryTrainer

nltk.download('all')

     |████████████████████████████████| 2.9 MB 16.3 MB/s 
     |████████████████████████████████| 270 kB 82.7 MB/s 
     |████████████████████████████████| 981 kB 81.1 MB/s 
     |████████████████████████████████| 78 kB 9.7 MB/s 
     |████████████████████████████████| 636 kB 79.8 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.6 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 125 kB 67.0 MB/s 
     |████████████████████████████████| 243 kB 68.2 MB/s 
     |████████████████████████████████| 1.3 MB 76.6 MB/s 
     |████████████████████████████████| 1.2 MB 84.1 MB/s 
     |████████████████████████████████| 271 kB 86.8 MB/s 
     |████████████████████████████████| 160 kB 76.8 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=4ecae35596464a79ab3d49f0ee8ab9e9dec7326419abd946df1f247bd07e8691
  Stored in directory: /root/.cac

True

In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
#import torch_xla
#import torch_xla.core.xla_model as xm

#dev = xm.xla_device()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Split Content into sentences

In [4]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
#data = data[['CONTENT']].dropna()
print(f'All records: {len(data)}')
df_sentences = pd.DataFrame()
df_sentences['sequence'] = data['CONTENT'].apply(lambda x: nltk.sent_tokenize(x, language='german'))
df_sentences = df_sentences['sequence'].apply(lambda x: pd.Series(x)).stack().reset_index(drop=True).to_frame('sequence')

too_short = df_sentences[df_sentences['sequence'].str.split(' ').str.len() < 5]
too_long = df_sentences[df_sentences['sequence'].str.split(' ').str.len() > 32]
print(f'Sentences: {len(df_sentences)}')
print(f'Too short: {len(too_short)}, {len(too_short) / len(df_sentences)}')
print(f'Too long: {len(too_long)}, {len(too_long) / len(df_sentences)}')

All records: 77381


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


Sentences: 2710107
Too short: 94863, 0.035003414994315724
Too long: 187229, 0.06908546415326038


In [ ]:
d = data.iloc[:15000]
dataset = SentenceClusterDataset(tokenizer, d)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (6). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (11). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:248: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (6). Possibly due to duplicate points in X.
/usr/local/lib/python3.7/dist-packages/

ValueError: ignored

In [ ]:
test_size = int(len(dataset) * 0.05)
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))

with open('/content/drive/MyDrive/datasets/train_set-1.pt', mode='wb') as f:
  pickle.dump(train_set, f)

with open('/content/drive/MyDrive/datasets/test_set-1.pt', mode='wb') as f:
  pickle.dump(test_set, f)


109861


In [ ]:
## Train with reconstruction and similarity loss

encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = EncoderDecoder(encoder_config=encoder_config, decoder_config= decoder_config)
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=15, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-2', 
                         tensorboard_path='/content/drive/MyDrive/runs',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 109850/109850 [4:44:45<00:00,  6.43steps/s, loss=3.78]



Epoch Loss: 3.778007705461613
Epoch Time Elapsed: 17085.448076963425


100%|██████████| 5781/5781 [11:40<00:00,  8.25steps/s, val_loss=2.59]



Validation Loss: 2.586546724800395
Rouge Scores: {'rouge-1': {'r': 0.500825660961373, 'p': 0.4869161342490429, 'f': 0.4924033471586399}, 'rouge-2': {'r': 0.28234287322529533, 'p': 0.27907397312808374, 'f': 0.2799578017214118}, 'rouge-l': {'r': 0.4725052326286401, 'p': 0.4591232422050198, 'f': 0.464418421366668}}
Running Epoch 2 ...


...

Epoch 4: 100%|██████████| 109850/109850 [4:42:52<00:00,  6.47steps/s, loss=2.15]



Epoch Loss: 2.1458220366864067
Epoch Time Elapsed: 70261.45773911476


100%|██████████| 5781/5781 [11:37<00:00,  8.29steps/s, val_loss=1.67]



Validation Loss: 1.6650880398461667
Rouge Scores: {'rouge-1': {'r': 0.6352030975805679, 'p': 0.6236809361954887, 'f': 0.6287924713078753}, 'rouge-2': {'r': 0.4522840560308082, 'p': 0.4472493241429784, 'f': 0.4493458819571791}, 'rouge-l': {'r': 0.6130115682375039, 'p': 0.6021395068442915, 'f': 0.6069593424056403}}
Running Epoch 5 ...


Epoch 5:  71%|███████   | 78003/109850 [3:20:26<1:20:42,  6.58steps/s, loss=2.01]

In [6]:
with open('/content/drive/MyDrive/datasets/sent-cluster/train_set-1.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/sent-cluster/test_set-1.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [9]:
len(test_set)+len(train_set)

115643

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-2/checkpoint-epoch-12')
encoder_config = checkpoint['encoder_config']
decoder_config = checkpoint['decoder_config']
try: 
  model
except NameError:
  model = EncoderDecoder(encoder_config=encoder_config, decoder_config=decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])
model.to('cuda:0')
try:
  optimizer
except NameError:
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}

trainer = SummaryTrainer(model, optimizer, criterions, epochs=4, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-2', 
                         tensorboard_path='/content/drive/MyDrive/runs',
                         tokenizer=tokenizer,
                         fp16=True)


Using device: cuda


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=12)

Running Epoch 13 ...


Epoch 13:  54%|█████▎    | 58839/109861 [2:30:16<2:38:10,  5.38steps/s, loss=1.47]

In [ ]:
test = {
    'input_ids': train_set.dataset.X_enc[0]['input_ids'][0:1]
}
print(tokenizer.decode(test['input_ids'][0], skip_special_tokens=True))
with torch.no_grad():
  generated = model.reconstruct(test, n_beams=5, return_all_candidates=False)
  decoded = tokenizer.decode(generated[0][0], skip_special_tokens=True)
  print(decoded)
  #rouge = Rouge()
  #ref = tokenizer.decode(inp['input_ids'][0], skip_special_tokens=True)
  #scores = rouge.get_scores(test_sentence, decoded, avg=True)

Einerseits wolle man die unter Bidens Vorgänger Donald Trump beschädigten Beziehungen zu Deutschland kitten.
Einerseits wolle man die unter Bidens Vorgänger Donald Trump beschädigten Beziehungen zu Deutschland kitten.


In [ ]:
test_summary = train_set.dataset.X_enc[1]
print(tokenizer.batch_decode(test_summary['input_ids'], skip_special_tokens=True))
with torch.no_grad():
  generated = model.summarize_sequences(test_summary)
  print(tokenizer.decode(generated[0][0], skip_special_tokens=True))

['Nach regierungsinternen Diskussionen sei das aber revidiert worden.', 'Ein hochrangiger Regierungsvertreter sagte nach Angaben von Politico, man befinde sich in einer schwierigen Lage.', 'Daran hat auch der Regierungswechsel in Washington im Januar nichts geändert.']
Es sei die Regierungserklärung macht die Lager nach Sachsen - Anhalt über Geimpft.


In [ ]:
generated

(tensor([[    3,   482,   350,    30,  3693,  8480,  2241,    30,  4458,   188,
           4267, 26935,  6325,   204, 11242, 26911,  4090, 26914,     4,     4]],
        device='cuda:0'), 20.766192579122016)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

### Training the DocSplitModel 
Split Documents into n(=4) sub-parts and train summary on those

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
#data = data[['CONTENT']].dropna()
# take only first 5000 articles
data = data.iloc[:30000]

In [ ]:
dataset = DocSplitDataset(tokenizer, data)

In [ ]:
## Train with reconstruction and similarity loss

encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = EncoderDecoder(encoder_config=encoder_config, decoder_config= decoder_config)
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=4, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4', 
                         tensorboard_path='/content/drive/MyDrive/runs',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
test_size = int(len(dataset) * 0.1)
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))

with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='wb') as f:
  pickle.dump(train_set, f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='wb') as f:
  pickle.dump(test_set, f)

26150


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 26150/26150 [7:39:51<00:00,  1.06s/steps, loss=4.73]



Epoch Loss: 4.7305673095857435
Epoch Time Elapsed: 27591.78464460373


100%|██████████| 2905/2905 [51:38<00:00,  1.07s/steps, val_loss=3.92]



Validation Loss: 3.919837644252827
Rouge Scores: {'rouge-1': {'r': 0.3166369617322374, 'p': 0.17914816776788217, 'f': 0.21615398664013216}, 'rouge-2': {'r': 0.0776315336593444, 'p': 0.05181684141344359, 'f': 0.058119195074027574}, 'rouge-l': {'r': 0.29804285118671914, 'p': 0.16755867307085923, 'f': 0.2030863797536572}}
Summary Rouge Scores: {'rouge-1': {'r': 0.26701504483143157, 'p': 0.14165953883347437, 'f': 0.17569920960566185}, 'rouge-2': {'r': 0.03455558991476699, 'p': 0.018980894549399066, 'f': 0.0228804369555645}, 'rouge-l': {'r': 0.24511605208213028, 'p': 0.13003552195179838, 'f': 0.16119922910726273}}
Running Epoch 2 ...


Epoch 2: 100%|██████████| 26150/26150 [7:46:21<00:00,  1.07s/steps, loss=4.02]



Epoch Loss: 4.0239081680816
Epoch Time Elapsed: 27981.39450263977


100%|██████████| 2905/2905 [50:54<00:00,  1.05s/steps, val_loss=3.64]



Validation Loss: 3.641436288752921
Rouge Scores: {'rouge-1': {'r': 0.3471180279237308, 'p': 0.21094787822294922, 'f': 0.24924869738055816}, 'rouge-2': {'r': 0.099118246416245, 'p': 0.07047453802817699, 'f': 0.07755666364475682}, 'rouge-l': {'r': 0.327425025377651, 'p': 0.19892175400721152, 'f': 0.23487516059473074}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2700024787455477, 'p': 0.15493735322804908, 'f': 0.18655349030407503}, 'rouge-2': {'r': 0.03771546902917915, 'p': 0.022240918293997027, 'f': 0.02613624119300399}, 'rouge-l': {'r': 0.24657958437772357, 'p': 0.1414226068681972, 'f': 0.17019898782156703}}
Running Epoch 3 ...


Epoch 3:  85%|████████▌ | 22245/26150 [6:32:21<1:16:15,  1.17s/steps, loss=3.83]

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4/checkpoint-epoch-6')
encoder_config = ModelConfig(checkpoint['encoder_config'])
decoder_config = ModelConfig(checkpoint['decoder_config'])
try: 
  model
except NameError:
  model = EncoderDecoder(encoder_config=encoder_config, decoder_config=decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])
model.to('cuda:0')
try:
  optimizer
except NameError:
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}

trainer = SummaryTrainer(model, optimizer, criterions, epochs=2, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4/metrics',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=6)

Running Epoch 7 ...


Epoch 7: 100%|██████████| 26150/26150 [7:43:40<00:00,  1.06s/steps, loss=3.51]



Epoch Loss: 3.513296066947837
Epoch Time Elapsed: 27820.069763183594


100%|██████████| 2905/2905 [51:08<00:00,  1.06s/steps, val_loss=3.25]



Validation Loss: 3.2475408328176383
Rouge Scores: {'rouge-1': {'r': 0.39884627626200003, 'p': 0.27299461099875627, 'f': 0.3104760125554185}, 'rouge-2': {'r': 0.14947930085647698, 'p': 0.11316435851070071, 'f': 0.12227603711666014}, 'rouge-l': {'r': 0.3759568898706281, 'p': 0.2575603190545017, 'f': 0.2925998231589457}}
Summary Rouge Scores: {'rouge-1': {'r': 0.27823172219694536, 'p': 0.17582801489775313, 'f': 0.20497415246422604}, 'rouge-2': {'r': 0.03951560380794148, 'p': 0.026157040627729045, 'f': 0.02949598176214541}, 'rouge-l': {'r': 0.25205282275505536, 'p': 0.1590677049799588, 'f': 0.185427064437369}}
Running Epoch 8 ...


Epoch 8: 100%|██████████| 26150/26150 [7:44:03<00:00,  1.06s/steps, loss=3.47]



Epoch Loss: 3.467924015433394
Epoch Time Elapsed: 27843.16139650345


100%|██████████| 2905/2905 [51:08<00:00,  1.06s/steps, val_loss=3.21]



Validation Loss: 3.20629877881651
Rouge Scores: {'rouge-1': {'r': 0.40621156084584986, 'p': 0.2793062887309845, 'f': 0.31910413313419195}, 'rouge-2': {'r': 0.1585779506963309, 'p': 0.12007021413516192, 'f': 0.13088965793177715}, 'rouge-l': {'r': 0.37914069630593605, 'p': 0.26187057425365745, 'f': 0.2984654705360201}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2796421884606189, 'p': 0.17723157173047357, 'f': 0.20627060082199955}, 'rouge-2': {'r': 0.04031700335680554, 'p': 0.02654222590854046, 'f': 0.0300914626980606}, 'rouge-l': {'r': 0.25377045126244063, 'p': 0.16055749616376552, 'f': 0.18690630412753462}}
Total Time Elapsed: 62063.13387441635


# Use pre-trained bert-2-bert embedding matrix

In [ ]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_pretrained('./drive/MyDrive/checkpoints/bert2bert/reconstruct-content/v3/checkpoint-4500')

encoder_bert = bert2bert.encoder
decoder_bert = bert2bert.decoder.bert

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()
# take only first 5000 articles
data = data.iloc[:30000]

In [ ]:
with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [ ]:
encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": False,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": False,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = EncoderDecoder(encoder_config=encoder_config, decoder_config= decoder_config, bert_models=(encoder_bert, decoder_bert))
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=4, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-5', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-5/metrics',
                         tokenizer=tokenizer,
                         fp16=True)

Using device: cuda


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 26150/26150 [10:14:26<00:00,  1.41s/steps, loss=4.82]



Epoch Loss: 4.824425172658769
Epoch Time Elapsed: 36866.658673763275


100%|██████████| 2905/2905 [1:10:46<00:00,  1.46s/steps, val_loss=3.97]



Validation Loss: 3.9749043764675203
Rouge Scores: {'rouge-1': {'r': 0.33433560546220586, 'p': 0.17268660940385921, 'f': 0.2167767543816698}, 'rouge-2': {'r': 0.08045219590697052, 'p': 0.046409035513348594, 'f': 0.05533405488571573}, 'rouge-l': {'r': 0.3151299995120059, 'p': 0.1629240533965102, 'f': 0.20437772917775585}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2776484853539511, 'p': 0.13576149410294575, 'f': 0.17274788841785202}, 'rouge-2': {'r': 0.037098675465559255, 'p': 0.01876346953525056, 'f': 0.023265123821502146}, 'rouge-l': {'r': 0.2561096911138283, 'p': 0.12519215801562508, 'f': 0.15920925764330476}}
Running Epoch 2 ...


Epoch 2: 100%|██████████| 26150/26150 [10:52:04<00:00,  1.50s/steps, loss=4.07]



Epoch Loss: 4.073721865387224
Epoch Time Elapsed: 39124.583790779114


100%|██████████| 2905/2905 [1:11:52<00:00,  1.48s/steps, val_loss=3.68]
